#### Imports

In [2]:
import tweepy
import pandas as pd
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import urllib
import requests
import wikipedia
import urllib
import random
import bs4
import datetime
import schedule 
import time 
import re
import os
from imgurpython import ImgurClient
from datetime import date
import api_keys

#### Authorization

In [3]:
auth = tweepy.OAuthHandler(api_keys.twitter.CONSUMER_KEY, api_keys.twitter.CONSUMER_SECRET)
auth.set_access_token(api_keys.twitter.ACCESS_KEY,api_keys.twitter.ACCESS_SECRET)
api = tweepy.API(auth)
try:
    redirect_url = auth.get_authorization_url()
    print('Authenticated!')
except tweepy.TweepError:
    print('Error! Failed to get request token.')

Authenticated!


#### Colors

In [4]:
class colors_rgb:
    #https://www.schemecolor.com/separate-hearts.php
    twitter_color = (56, 161, 243)
    black = (0, 0, 0)
    desaturated_cyan = (115,154,159)
    twilight_lavender =  (145, 69, 102)
    old_rose = (195, 123, 129)
    burlywood = (231, 183, 128)
    teal_blue = (50, 108, 132)

### Parent Class

In [5]:
class post:
    stopwords = list(pd.read_csv('stopwords.csv')['stopwords'])
    
    def __init__(self,astr):
        
        self.astr = astr
        self.url = ""
        self.has_image = False
        self.stopwords = list(pd.read_csv('stopwords.csv')['stopwords'])
        self.content = ''
        
    def string_to_img(self,font_style,bgcolor,justify='left',save=True,verbose=False):    
        '''
        Converts string into an image
        '''
        from PIL import Image, ImageDraw, ImageFont
        import textwrap

        astr = self.astr
        astr_len = len(astr)
        astr_spaces = astr.count(' ')
        astr_words = astr_spaces + 1
        ave_asrt_letters_per_word = (astr_len - astr_spaces)/astr_words

        words = astr.split(" ")
        len_words = list(map(len,words))


        width = 11

        if astr_words <= 4:
            width = max(len_words)
        elif astr_words >= 8:
            len_words.sort()
            width = max(len_words) + len_words[-2]

        para = textwrap.wrap(astr, width=width)

        MAX_W, MAX_H = 520, 290

        font_calc_constant = .80
        font_size = int(np.sqrt((MAX_W * MAX_H)/((font_calc_constant**2)*astr_len)))

        max_len_line = max(list(map(len,para)))
        lines = len(para)

        if font_size*font_calc_constant*max_len_line > MAX_W:
            font_size = int(MAX_W/((font_calc_constant**3)*max_len_line))

        if font_size*lines > MAX_H:
            font_size = int(MAX_H/lines)

        im = Image.new('RGB', (MAX_W, MAX_H), bgcolor)
        draw = ImageDraw.Draw(im)
        font = ImageFont.truetype(font_style,font_size)

        line_num = 0
        num_of_lines = len(para)
        top_pad = ((MAX_H - (font_size*num_of_lines*.9))/(num_of_lines))/2#0

        for line in para:
            w, h = draw.textsize(line, font=font)
            if justify=='center':
                draw.text(((MAX_W - w) / 2, ((MAX_H/num_of_lines)*line_num) +top_pad), line, font=font)

            elif justify=='left':
                draw.text((10, ((MAX_H/num_of_lines)*line_num) +top_pad), line, font=font)

            line_num +=1

        if verbose==True:
            print('string length: ' + str(astr_len))
            print('string words: ' + str(astr_words))
            print('ave letters per word: ' + str(ave_asrt_letters_per_word))
            print('max length of words: ' + str(max(len_words)))
            print('width: ' + str(width))
            print('font size: ' + str(font_size))
            print('num_of_lines: ' + str(num_of_lines))
  
        im.save('tweet_image.jpg')
            
        self.image = im
        self.has_image = True

    
    def clean_add_hash(self):
        astr = self.astr
        import re
        astr = re.sub('[^A-Za-z0-9]+', ' ', astr).lower()
        astrwords = astr.split()
        resultwords  = [word for word in astrwords if word.lower() not in self.stopwords]
        resultwords  = ['#'+word for word in resultwords]
        resultwords = list(set(resultwords))
        result = ' '.join(resultwords)
        
        self.hashtags = result
        
        
    def popular_hashtags(self,place = 23424977, api = api,verbose = True):
        tags = api.trends_place(place) 
        
        popular_hashtags_df = pd.DataFrame(tags[0]['trends']).sort_values('tweet_volume',ascending=False)[['name','tweet_volume']].head(10).reset_index().drop('index',axis=1)
        if verbose == True:
            print(popular_hashtags_df)
        self.popular_hashtags_str = popular_hashtags_df.loc[0,'name']
        
    
    def find(select,index,start,end,soup):
        length = len(start)
        html_code = str(soup.select(select)[index])
        target_start = html_code.find(start) + length
        target_end = html_code.find(end,target_start)
        target = html_code[target_start:target_end]
        return target 
    
    def image_resize(self):
        import os
        if self.has_image == True:
            #import PIL
            from PIL import Image
            if os.stat('tweet_image.jpg').st_size>3000:
                im = Image.open('tweet_image.jpg')
                width, height = im.size
                aspect_ratio = width/height
                
                if (height>290):
                    new_height = 290
                    new_width = (width*new_height)/height
                
                else:
                    new_width = 520
                    new_height = (new_width*height)/width
                
                
                
                
                
                im = im.resize((int(new_width),int(new_height)),Image.ANTIALIAS)
                im.save('tweet_image.jpg',quality=95)
                print('image resized!')

        
        
    def tweet(self,api = api, paragraph=''):
        print('time: ' + datetime.datetime.now().strftime("%Y-%m-%d %H:%M"))
        if self.content == '': 
            self.content = self.astr+ "\n" + paragraph + "\n" +self.hashtags + "\n" + self.url
        print(self.content)
        print('Post Length: ' + str(len(self.content)))
        counter = 0
        while counter <3:
            try:
                if self.has_image == True:
                    print('has image')
                    _ = api.update_with_media('tweet_image.jpg',self.content)
                else:
                    print('no image')
                    _ = api.update_status(self.content)
                break
                print(tweet.content)
                print()
                print()
                
            except tweepy.TweepError as e:      
                print(e)
                if e.api_code == 187:
                    print('Try next index or wait for update')
                    break
                else:
                    print('Attempt: ' + str(counter))
                counter = counter + 1
                
        print()
        print()


        



#### News API

In [6]:
class news_api(post):

    def __init__(self,astr='',index=0):
        super().__init__(index)
        self.topic = astr
        self.index = index
    
    def requests_get(self):
        
        if self.topic == '':
            url = ('https://newsapi.org/v2/top-headlines?'
                   'country=us&'
                   'apiKey=' + api_keys.news_api.api_key)
            
        elif self.topic != '':
            url = ('https://newsapi.org/v2/everything?'
                   'q=' + self.topic + '&'
                   'sortBy=popularity&'
                   'apiKey='+ api_keys.news_api.api_key)
            
        response = requests.get(url)
        
        self.astr = response.json()['articles'][self.index]['title']
        self.title = self.astr
        self.img = response.json()['articles'][self.index]['urlToImage']
        self.url = response.json()['articles'][self.index]['url']
        
        


### Youtube

In [7]:
class youtube(post):

    def __init__(self,index=0):
        super().__init__(index)
        self.index = index
    
    def requests_get(self):
        
        res = requests.get('https://www.youtube.com/feed/trending')
        res.raise_for_status()
        soup = bs4.BeautifulSoup(res.text,'html.parser')
        
        
        self.top_video = post.find(select = "div h3",index = self.index + 3,start = 'title="',end = '">',
                                    soup = soup)
        
        
        self.user = post.find(select = "div a",index = self.index + 21,start = '"/user/',
                                             end = '">',soup = soup)
        
        self.url = post.find(select = "div h3",index = self.index + 3,start = 'href="',
                                             end = '" title=', soup = soup)
        self.url = "https://www.youtube.com" + self.url
        
        self.astr = self.top_video + ' ' + self.user
        
        

### Famous Birthdays

In [8]:
class birthdays(post):
    def __init__(self):
        super().__init__(astr="")
        
    def requests_get(self,topic = ''):
        
        now = datetime.datetime.now()
        date_now = now.strftime("%B-%d")
        date_now = date_now.lower()
        
        res = requests.get('https://www.thefamousbirthdays.com/'+date_now)

        res.raise_for_status()
        soup = bs4.BeautifulSoup(res.text,'html.parser')
        
        self.celebrant = post.find(select = "div a",index = 14,start = '">',end = '(',
                                    soup = soup)
        self.celebrant = self.celebrant.strip()
        
        self.age = post.find(select = "div a",index = 14,start = '(',
                                end = ')',soup = soup)
        self.age = self.age.strip()
        if len(self.age)>2:
            self.age = ''
            
        self.astr = 'Happy Birthday! ' + self.celebrant
        


### NASA

In [9]:
class nasa(post):
    def __init__(self):
        super().__init__(astr="")
        
    def requests_get(self):
        res = requests.get("https://api.nasa.gov/planetary/apod?api_key="+api_keys.nasa.api_key)
        self.url = res.json()['url']
        self.title = res.json()['title']
        
        self.astr = self.title
        
        if 'youtube' in self.url:
            self.url = self.url.replace('embed/', 'watch?v=')
        else:
            self.has_image = False
            try:
                urllib.request.urlretrieve(self.url, "tweet_image.jpg")
                self.has_image = True

            except:
                pass
        

### Wikipedia

In [10]:
class wiki(post):
    def __init__(self):
        super().__init__(astr="")
        
    def num_sentences(topic,topic_page,title,sentences):

        summary = wikipedia.summary(topic, sentences) 
        summary = re.sub("[\(\[].*?[\)\]]", "", summary)
        url = topic_page.url
        astr = title 
        total_len = len(astr) + astr.count(' ')+1 + len(url) + len(summary)
        
        return total_len,summary,url,astr
    
    def requests_get(self):

        try:
            unusual_articles = wikipedia.page("Wikipedia:Unusual articles")
        except tweepy.TweepError as e:      
            print(e)
            
        topic = unusual_articles.links[random.randint(1,len(unusual_articles.links))]
        
        counter = 0
        while counter<3:
            try:
                topic_page = wikipedia.page(topic)
                time.sleep(1)
                break
            except wikipedia.exceptions as e:
                print(e)
                counter = counter + 1
                pass
            
        self.title = topic.title()
         
        summary_length,self.summary,self.url,self.astr = wiki.num_sentences(topic,topic_page,self.title,5)
        
        self.title = self.astr
        
        self.has_image = False

        try:
            
            if str(topic_page.images[0][-4:]) == '.png':
                urllib.request.urlretrieve(topic_page.images[0], "tweet_image.png")
                Image.open('tweet_image.png').convert('RGB').save('tweet_image.jpg')
            else:
                urllib.request.urlretrieve(topic_page.images[0], "tweet_image.jpg")
                   
            self.has_image = True
            
            print('Image Type: ' + str(topic_page.images[0][-4:]))
            if topic_page.images[0][-4:] not in ['.png','.jpg','.jpeg']:
                self.has_image = False
            

        except:
            print('No image')
            
        

# Imgur

In [11]:
class Imgur(post):
    
    def __init__(self):
        super().__init__(astr="")
        
    def requests_get(self,top_or_hot = 'top'):

        client = ImgurClient(api_keys.imgur.client_id, api_keys.imgur.client_secret)
        items = client.gallery(section=top_or_hot, sort='best', page=0, window='day', show_viral=False)
        
        self.url = items[0].link
        self.title = items[0].title
        self.astr = self.title
        

# Schedule Tweet Function

In [ ]:
class Schedule_tweet_function:
    
    def payday(date_dict):
        astr = ''
        if (date_dict['day'] == 1) & (date_dict['dow'] in ['Mon','Tue','Wed','Thu','Fri']):
            astr = 'Happy Payday!'
        elif (date_dict['day'] == 15) & (date_dict['dow'] in ['Mon','Tue','Wed','Thu','Fri']):
            astr = 'Happy Payday!'
        return astr
    
    def holiday(date_dict):
        astr= ''
        if (date_dict['month'] == 'Jan') & (date_dict['day'] == 1):
            astr = 'Happy New Year!'

        elif (date_dict['month'] == 'Jun') & (date_dict['day'] == 4):
            astr = 'Happy Independence Day!'

        elif (date_dict['month'] == 'Dec') & (date_dict['day'] == 25):
            astr = 'Merry Christmas!'

        elif (date_dict['month'] == 'Jan') & (15<=date_dict['day']) & (date_dict['day']<=21) & (date_dict['dow']== 'Mon'):
            astr = 'Happy MLK Day!'

        elif (date_dict['month'] == 'Feb') & (15<=date_dict['day']) & (date_dict['day']<=21) & (date_dict['dow'] == 'Mon'):
            astr = 'Happy Presidents Day!'

        elif (date_dict['month'] == 'May') & (24<date_dict['day']) & (date_dict['dow'] == 'Mon'):
            astr = 'Happy Memorial Day!'

        elif (date_dict['month'] == 'Sep') & (date_dict['day']<=7) & (date_dict['dow'] == 'Mon'):
            astr = 'Happy Labor Day!'

        elif (date_dict['month'] == 'Oct') & (8<=date_dict['day']) & (date_dict['day']<=14)& (date_dict['dow'] == 'Mon'):
            astr = 'Happy Columbus Day!'

        elif (date_dict['month'] == 'Nov') & (date_dict['day'] == 11):
            astr = 'Happy Veterans Day!'

        elif (date_dict['month'] == 'Nov') & (22<=date_dict['day']) & (date_dict['day']<=28) & (date_dict['dow'] == 'Thu'):
            astr = 'Happy Thanksgiving Day!'
            
        return astr
    
    
    def post(astr,font = 'helvetica.ttf',bgcolor = colors_rgb.teal_blue):
        tweet = post(astr)
        tweet.string_to_img(font, bgcolor)
        tweet.clean_add_hash()
        tweet.tweet()
        
    def daily_post():
        from datetime import date
        today = date.today()
        astr = ''
        font = 'helvetica.ttf'
        bgcolor = colors_rgb.teal_blue
        date_dict = {}

        date_dict['month'] = today.strftime("%b")
        date_dict['day'] = int(today.strftime("%d"))
        date_dict['dow'] = today.strftime("%a")
        print("Today's date: ", today)
        print("month: ", date_dict['month'])
        print("day: ", date_dict['day'])
        print("dow: ", date_dict['dow'])

        astr = Schedule_tweet_function.payday(date_dict)
        astr = Schedule_tweet_function.holiday(date_dict)
        
        if astr !='':
            tweet = post(astr)
            tweet.string_to_img(font, bgcolor)
            tweet.clean_add_hash()
            tweet.tweet()

        print('astr: ' + astr)
        
    def news_api(topic="",index=0):
        print('news api')
        tweet = news_api(topic,index)
        tweet.requests_get()
        tweet.clean_add_hash()
        tweet.tweet()
        
    def youtube(index=0):
        print('youtube')
        tweet = youtube(index)
        tweet.requests_get()
        tweet.clean_add_hash()
        tweet.hashtags = tweet.hashtags + ' #youtube' 
        tweet.tweet()

        
    def birthdays(font = 'helvetica.ttf',bgcolor = colors_rgb.teal_blue):
        print('birthdays')
        tweet = birthdays()
        #tweet.string_to_img(font, bgcolor)
        tweet.requests_get()
        tweet.clean_add_hash()
        tweet.hashtags = tweet.hashtags + ' #happybirthday' 
        tweet.string_to_img(font, bgcolor)
        tweet.tweet()
        
    def nasa():
        print('nasa')
        tweet = nasa()
        tweet.requests_get()
        tweet.clean_add_hash()
        tweet.hashtags = tweet.hashtags + ' #nasa' 
        tweet.image_resize()
        tweet.content = tweet.astr+ "\n" + tweet.hashtags
        tweet.tweet()

    def that_wikipedia():
        print('wiki')
        import os
        tweet = wiki()
        tweet.requests_get()
        tweet.clean_add_hash()
        tweet.hashtags = tweet.hashtags + ' #wikipedia #unusualarticles'
        tweet.image_resize()
        summary_len = 280 - len(tweet.url) - len(tweet.hashtags) - 6
        tweet.summary = tweet.summary[:summary_len] + ' ...'
        tweet.content = tweet.summary + "\n" +tweet.url + "\n" + tweet.hashtags
        tweet.tweet()
        

    def imgur(top_or_hot = 'top'):
        print('imgur')
        tweet = Imgur()
        tweet.requests_get(top_or_hot)
        tweet.clean_add_hash()
        tweet.hashtags = tweet.hashtags + ' #imgur #meme' 
        tweet.tweet()
        
        
        


In [ ]:
import schedule
schedule.every(1.5).hours.do(lambda: Schedule_tweet_function.news_api()) 
schedule.every(1.1).hours.do(lambda: Schedule_tweet_function.youtube(1))
schedule.every(4).hours.do(lambda: Schedule_tweet_function.imgur('top'))
schedule.every(1).hours.do(lambda: Schedule_tweet_function.imgur('hot'))
schedule.every().day.at("04:15").do(Schedule_tweet_function.daily_post) 
schedule.every().day.at("08:31").do(Schedule_tweet_function.birthdays) 
schedule.every().day.at("08:35").do(Schedule_tweet_function.nasa)
schedule.every().day.at("08:40").do(lambda: Schedule_tweet_function.news_api('mcu'))
schedule.every().day.at("07:40").do(lambda: Schedule_tweet_function.news_api('stocks'))
schedule.every().day.at("04:50").do(lambda: Schedule_tweet_function.news_api('yang'))
schedule.every().day.at("13:00").do(lambda: Schedule_tweet_function.news_api('tv'))
schedule.every().day.at("09:10").do(lambda: Schedule_tweet_function.post('Sieze the day!'))
schedule.every().friday.at("07:00").do(lambda: Schedule_tweet_function.post('Happy Friday!!')) 

schedule.every().hour.do(Schedule_tweet_function.that_wikipedia)

while True:   
    schedule.run_pending() 
    time.sleep(30) 
    
    